<img src="img/mioti.png" >


# Proyecto Reconocimiento Facial: Detector

<img src="./img/emociones.png" style="width: 800px">

### Objetivos

* En este notebook vamos a probar la combinación de los diferentes modelos generados anteriormente. El objetivo es que en una grabación en tiempo real, el sistema sea capaz de poner el nombre de la persona identificada, si esta ha sido registrada, en caso contrario será 'desconocido' y la emoción de esta persona, esto se realizará para todos los rostros identificados, sean conocidos por el modelo o no

### Importación de librerías

* Para poner en práctica nuestro reconocedor utilizaremos estas librerías:
    * cv2: para trabajar con las imágenes de video y aplicar el reconocedor de personas
    * os: para trabajar con directorios
    * numpy: para realizar operaciones matemáticas
    * tensorflow: para definir modelos

# Prepación del entorno

In [ ]:
# !pip install opencv-contrib-python

In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array

print(cv2.__version__)

4.5.5


# Carga de los modelos

* Vamos a cargar nuestros modelos con los ficheros json y h5 generados durante el entrenamiento de la red de emociones para nuestro modelo de emociones y el fichero xlm para cargar nuestro modelo de reconocimiento de personas

* Después en nuestra función face_detector vamos a incluir las predicciones realizadas por nuestros modelos

* Los modelos siempre van a devolver una clase en cada predicción, en el caso de las emociones esto no genera ningún problema ya que queremos identificar el estado de animo, sin embargo en el caso de las personas no siempre vamos a conocer a quién estamos midiendo, para evitar el caso en que a una persona desconocida se le asigne el nombre de alguien registrado vamos a establecer un valor máximo a un umbral y cualquier valor que se encuentre por encima de este hará que esa persona sea clasificada como 'desconocido', en nuestro caso hemos establecido ese umbral por debajo de 10000 tras realizar diferentes pruebas

In [2]:
# Carga del modelo Haarcascade.
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [3]:
#Carga del modelo de detección de emociones.

file_model="./model/vgg-face-model-v4.json"
file_weights="./model/vgg-face-v4.h5"
class_labels=['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [4]:
json_file = open(file_model, 'r')
model_json = json_file.read()
json_file.close()
reconocedor_emociones = tf.keras.models.model_from_json(model_json)
reconocedor_emociones.load_weights(file_weights)

In [6]:
%%time
# Carga del modelo detección de personas.
reconocedor_facial=cv2.face.EigenFaceRecognizer_create()
reconocedor_facial.read('./model/modelo_reconocimiento_caras.xml')

Wall time: 47.7 s


In [7]:
data_path='./reconocimiento'
lista_gente=os.listdir(data_path) 
lista_gente=['Javier', 'Juan Pedro', 'Ricardo']
print(lista_gente)

['Javier', 'Juan Pedro', 'Ricardo']


In [8]:
#Función que visualiza los textos en las imagenes.
def draw_text_with_backgroud(img, text, x, y, font_scale, thickness=1, font=cv2.FONT_HERSHEY_SIMPLEX,
                            background=(175,50,200), foreground=(255,255,255), box_coords_1=(-5,5), box_coords_2=(5,-5)):
    (text_width, text_height) = cv2.getTextSize(text, font, fontScale=font_scale, thickness=1)[0]
    box_coords = ((x+box_coords_1[0], y+box_coords_1[1]), (x + text_width + box_coords_2[0], y - text_height + box_coords_2[1]))
    cv2.rectangle(img, box_coords[0], box_coords[1], background, cv2.FILLED)
    cv2.putText(img, text, (x, y), font, fontScale=font_scale, color=foreground, thickness=thickness)


In [9]:
#Función que orquesta, apoyandose en los tres modelos anteriores:
#     Captura del rostro - face_classifier.
#     Reconocimiento de personas - reconocedor_facial
#     Reconocimiento de emociones - reconocedor_emociones
def face_detector(img):
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.1, 4)
    if len(faces) == 0:
        return (0,0,0,0), np.zeros((48,48), np.uint8), img
    
    for idx,face in enumerate(faces):
        x,y,w,h = face
    
        cv2.rectangle(img,(x,y),(x+w,y+h),(175,50,200),2)
        roi_gray = gray[y:y+h, x:x+w]
        #ROI para el modelo reconocedor de emociones.
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation = cv2.INTER_CUBIC)
        #ROI para el modelo reconocedor de personas.
        roi_gray_face = cv2.resize(roi_gray, (150, 150), interpolation = cv2.INTER_CUBIC)
        
        #Procesamos el roi
        if np.sum([roi_gray]) != 0.0:
            roi = roi_gray.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            
            #Predicción de persona
            result=reconocedor_facial.predict(roi_gray_face)
            
            #Predicción de emoción
            preds = reconocedor_emociones.predict(roi)[0]
            
            if result[1]<11000:
                label = f'Persona {lista_gente[result[0]]} EMOCION {class_labels[preds.argmax()]}'  
            else:
                label = f'Persona Desconocido EMOCION {class_labels[preds.argmax()]}'

            draw_text_with_backgroud(img, label, x + 5, y, font_scale=0.5)
            if (idx==1):
                draw_text_with_backgroud(img, label, x + 5, y, font_scale=0.5, background=(15,150,200), foreground=(255,255,255), box_coords_1=(-7,7), box_coords_2=(7,-7))
        else:
            draw_text_with_backgroud(img, "No Face Found", x + 5, y, font_scale=0.5)
   
            
    return (x,w,y,h), roi_gray, img

# Ejecución de ReconocIAS

## Tiempo real

* El modelo irá identificando a las personas y sus emociones captadas durante la reproducción de un video, pero sin guardar esos resultados

In [11]:
#Camera
cap = cv2.VideoCapture(0)
# SIN GUARDAR
while True:

    
    ret, frame = cap.read()
    rect, face, image = face_detector(frame)

    
    res = cv2.resize(image, dsize=(400,400),interpolation=cv2.INTER_CUBIC)
    cv2.namedWindow("ReconocIAs", cv2.WINDOW_NORMAL)
    cv2.imshow("ReconocIAs", res)
    
#     cv2.resizeWindow('Reconocedor de Emociones', 900, 900)
    if cv2.waitKey(1) == 13: #13 es la tecla Entrer\is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows() 


## Tiempo Real con Registro 

* El modelo va a identificar a las personas que aparezcan en el vídeo, que será grabado con los resultados obtenidos en formato mp4

In [13]:
#Entrada
#File camera o video.avi mp4

b_Camera=True
b_Avi_Format=True
if (b_Camera):
    cap = cv2.VideoCapture(0)
else:
    cap = cv2.VideoCapture('./images/videoEntrada.avi')

#SALIDA FORMATO AVI
if (b_Avi_Format):
    salida = cv2.VideoWriter('./images/videoSalida.avi',cv2.VideoWriter_fourcc(*'XVID'),6.0,(640,480))
#SALIDA FORMATO MP4
else:
    salida = cv2.VideoWriter('./images/videoSalida.mp4',cv2.VideoWriter_fourcc(*'mp4v'),20.0,(640,480)) 
    
while True:

    ret, frame = cap.read()
    rect, face, image = face_detector(frame)
    cv2.namedWindow("ReconocIAs", cv2.WINDOW_NORMAL)
    cv2.imshow('"ReconocIAs"', image)
    salida.write(image)

    if cv2.waitKey(1) == 13: #13 es la tecla Entrer\is the Enter Key
        break

cap.release()
salida.release()
cv2.destroyAllWindows() 


## Imagen Estática con Registro

* El modelo recoge una fotografía de un directorio dado la analiza y guarda los resultados en el mismo directorio

In [15]:
#File jpg
cap = cv2.imread('./images/imagenEntrada.jpg')
processed=False
while not processed:

    frame = cap
    rect, face, image = face_detector(frame)
  
    cv2.imwrite('./images/imagenSalida.jpg',image)
    processed=True
    if cv2.waitKey(1) == 13: #13 es la tecla Entrer\is the Enter Key
        break
              
cv2.destroyAllWindows() 

## Generación de video sin procesar.

* Graba un video en formato avi y lo guarda en un directorio sin realizar ningún tipo de reconocimiento

In [ ]:
captura = cv2.VideoCapture(0)
salida = cv2.VideoWriter('./images/videoSalida.avi',cv2.VideoWriter_fourcc(*'XVID'),20.0,(640,480))
while (captura.isOpened()):
  ret, imagen = captura.read()
  if ret == True:
    cv2.imshow('video', imagen)
    salida.write(imagen)
    if cv2.waitKey(1) == 13: #13 es la tecla Entrer\is the Enter Key
      break
  else: break
captura.release()
salida.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()
captura.release()